In [1]:
#default_exp exec.features

In [7]:
# export
from pathos.multiprocessing import ProcessingPool as Pool
import pathos
from montlake.atomgeom.features import get_features
from montlake.utils.utils import data_stream_custom_range
import pickle
import sys
import argparse 
import numpy as np
import itertools
import tqdm
import json

def load_data(dataset, name, outdir,numpoints):
    
    data = np.load(dataset)
    n = np.min((numpoints, data['R'].shape[0]))
    randomindices = np.random.choice(range(data['R'].shape[0]),n, replace = False)
    positions = data['R'][randomindices,:,:]
    
    np.save(outdir + '/'+name + 'indices',randomindices)
    np.save(outdir + '/'+name + 'positions',positions)
    
    return(positions)
    
def get_features_parallel(data,atoms2, atoms3,atoms4,cores):
    
    #cores = pathos.multiprocessing.cpu_count() - 1
    p = Pool(cores)    
    n = data.shape[0]
    print('n',n)
    results = list(tqdm.tqdm(p.imap(
        lambda j: get_features(data[j],
                               atoms2 = atoms2,
                               atoms3 = atoms3,
                               atoms4 = atoms4),
        data_stream_custom_range(list(range(n)))), total=n))
    output = {}
    output['features2'] = np.stack([results[i][0] for i in range(n)])
    output['features3'] = np.stack([results[i][1] for i in range(n)])
    output['features4'] = np.stack([results[i][2] for i in range(n)])
    return(output)

# def parse_args(args):
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--outdir")
#     parser.add_argument("--dataset")
#     parser.add_argument("--name")
#     parser.add_argument("--numpoints")
#     parser.add_argument("--cores")
#     return parser.parse_args(args)

def parse_args(args):
    parser= argparse.ArgumentParser()
    parser.add_argument("--config", help="Path to JSON config")
    args = parser.parse_args(args)
    return args

In [1]:
# # export

# try:
#     from nbdev.imports import IN_NOTEBOOK
# except:
#     IN_NOTEBOOK = False

# if __name__ == "__main__" and not IN_NOTEBOOK:
#     args = parse_args(sys.argv[1:])
#     dataset = args.dataset
#     name = args.name
#     numpoints = int(args.numpoints)
#     outdir = args.outdir
#     cores = int(args.cores)
#     data = load_data(dataset, name,outdir,numpoints)
#     natoms = data.shape[1]
    
#     if include_atoms2:
#         atoms2 = np.asarray(list(itertools.combinations(range(natoms), 2))) 
#     if include_atoms3:
#         atoms3 = np.asarray(list(itertools.combinations(range(natoms), 2))) 
#     if include_atoms4
#     atoms3 = np.asarray(list(itertools.combinations(range(natoms), 3))) 
#     atoms4 = np.asarray(list(itertools.combinations(range(natoms), 4))) 

#     output = get_features_parallel(data,atoms2,atoms3,atoms4,cores)
#     with open(outdir +'/'+ name + 'features', 'wb') as handle:
#         pickle.dump(output, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
# python -m montlake.exec.get_features --outdir ~/thesis_data/processed_data --numpoints 5 --name dimethylfuran --dataset ~/thesis_data/raw_data/dimethylfuran.DFT.PBE-TS.light.MD.500K.01.npz

In [4]:
#get_features(data[j],atoms2 = atoms2, atoms3 = atoms3,atoms4 = atoms4)

# dataset= '/Users/samsonkoelle/thesis_data/raw_data/dimethylfuran.DFT.PBE-TS.light.MD.500K.01.npz'
# data = load_data(dataset=dataset, name = 'asdf',outdir = '/Users/samsonkoelle',numpoints = 5)

In [5]:
# natoms = 15
# atoms2 = np.asarray(list(itertools.combinations(range(natoms), 2))) 
# atoms3 = np.asarray(list(itertools.combinations(range(natoms), 3))) 
# atoms4 = np.asarray(list(itertools.combinations(range(natoms), 4))) 

# output = get_features_parallel(data,atoms2, atoms3,atoms4)

n 5


100%|██████████| 5/5 [00:03<00:00,  1.53it/s]
